In [3]:
!pip install pandas
!pip install transformers 
!pip install sacremoses

  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.0 MB/s eta 0:00:0000:010:01


In [8]:
import pandas as pd
# Assuming the dataset is in CSV format
df = pd.read_csv("/kaggle/input/en-fr-translation-dataset/en-fr.csv")


In [9]:
df

,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English
...,...,...
22520371,Only with a highly overcompensatory stock–recr...,C'est seulement en cas de courbe stock–recrute...
22520372,The model predicts that the assumption made ab...,Le modèle prévoit que l'hypothèse émise au suj...
22520373,Overall the results confirm the unsatisfactory...,"Dans l'ensemble, les résultats confirment le p..."
22520374,"Error 404 — file not found Sorry, but the file...",Erreur 404 — fichier introuvable Nous sommes d...


In [10]:
# Read the first 1000 rows
df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv', nrows=1000)

In [11]:
from transformers import MarianMTModel, MarianTokenizer

def translate_to_french(text):
    model_name = 'Helsinki-NLP/opus-mt-en-fr'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    encoded_text = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

    translated = model.generate(encoded_text)

    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_text


english_text = "Hello, how are you?"
french_text = translate_to_french(english_text)
print(french_text)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Bonjour, comment allez-vous?


In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to translate an English sentence to French
def translate_to_french(sentence):
    translated_fr_tokens = []
    for word in sentence.split():
        tokens_fr = tokenizer_fr(word, return_tensors="pt")
        translated_fr = model_fr.generate(**tokens_fr)
        translated_fr_text = tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)[0]
        translated_fr_tokens.append(translated_fr_text)

    return ' '.join(translated_fr_tokens)

# Iterate through each row in the DataFrame and translate the 'English' column
for index, row in df.iterrows():
    english_sentence = row['English']
    translated_fr = translate_to_french(english_sentence)
    df.at[index, 'French_Translation'] = translated_fr

# Display the updated DataFrame
df

In [16]:
!pip install tqdm pandas 
import pandas as pd 
from tqdm import tqdm 
from transformers import MarianMTModel, MarianTokenizer

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Assuming df contains a column named 'en' with English sentences 

df['French_Translation'] = ''
# Function to translate an English sentence to French
def translate_to_french(sentence): 
    tokens_fr = tokenizer_fr(sentence, return_tensors="pt")     
    translated_fr = model_fr.generate(**tokens_fr)     
    translated_fr_text = tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)[0]     
    
    return translated_fr_text  

# Use tqdm with apply for a progress bar
tqdm.pandas(desc="Translating") 
df['French_Translation'] = df['en'].progress_apply(translate_to_french)  
# Display the updated DataFrame 
df.head()

Translating:  14%|█▍        | 144/1000 [02:06<12:32,  1.14it/s]


KeyboardInterrupt: 

In [18]:
import torch 
# Check if GPU is available and move the model to GPU
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

def batch_translate_to_french(sentences):
    tokens_fr = tokenizer_fr(sentences, return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)



In [21]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Move the model to GPU if available
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to batch translate English sentences to French
def batch_translate_to_french(sentences):
    tokens_fr = tokenizer_fr(sentences.tolist(), return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)

# Batch size for translation
batch_size = 8

# Iterate through each batch in the DataFrame and translate the 'English' the "en" column
for i in range(0, len(df), batch_size):
    batch_df = df.loc[i:i+batch_size-1]
    english_sentences = batch_df['en']
    
    # Translate the batch and update the DataFrame
    translated_fr_batch = batch_translate_to_french(english_sentences)
    df.loc[i:i+batch_size-1, 'French_Translation'] = translated_fr_batch

# Display the updated DataFrame
df


,en,fr,French_Translation
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...,Changing Lives (en anglais seulement) Changing...
1,Site map,Plan du site,Plan du site
2,Feedback,Rétroaction,Commentaires
3,Credits,Crédits,Crédits ouverts
4,Français,English,Français
...,...,...,...
995,A nova is a star that absorbs matter from a ne...,La matière absorbée finit par réchauffer l'éto...,Une nova est une étoile qui absorbe la matière...
996,It is a rare and spectacular event.,Il s'agit d'un phénomène rare et plutôt specta...,C'est un événement rare et spectaculaire.
997,"In 1977, he became the French editor of the Na...","En 1977, il devient l'éditeur francophone du N...","En 1977, il devient rédacteur en chef français..."
998,"In 1978, Lemay began the daunting task of asse...","En 1978, il entreprend la tâche colossale de r...","En 1978, Lemay commence la tâche redoutable d'..."


In [33]:
# Read the first 1000 rows
df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv', nrows=1000)

In [28]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
import multiprocessing

# Set the multiprocessing start method to 'spawn' for CUDA compatibility
multiprocessing.set_start_method('spawn', force=True)

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Move the model to GPU if available
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to translate English sentences to French
def translate_to_french(sentence):
    tokens_fr = tokenizer_fr(sentence, return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)[0]

# Translate each row in the DataFrame
df['French_Translation'] = df['en'].apply(translate_to_french)

# Display the updated DataFrame
print(df.head())


KeyboardInterrupt: 

In [29]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
import multiprocessing
import time

# Set the multiprocessing start method to 'spawn' for CUDA compatibility
multiprocessing.set_start_method('spawn', force=True)

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Move the model to GPU if available
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to translate English sentences to French
def translate_to_french(sentence):
    tokens_fr = tokenizer_fr(sentence, return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)[0]

# Measure the execution time
start_time = time.time()

# Translate each row in the DataFrame
df['French_Translation'] = df['en'].apply(translate_to_french)

# Calculate and print the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

# Display the updated DataFrame
print(df.head())


KeyboardInterrupt: 

In [32]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
import time

# Load a smaller English-French translation model (MarianMT)
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Move the model to GPU if available
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to batch translate English sentences to French
def batch_translate_to_french(sentences):
    tokens_fr = tokenizer_fr(sentences.tolist(), return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)

# Experiment with different batch sizes
batch_sizes = [8, 16, 32]
for batch_size in batch_sizes:
    start_time = time.time()

    # Iterate through each batch in the DataFrame and translate the 'English' column
    for i in range(0, len(df), batch_size):
        batch_df = df.loc[i:i+batch_size-1]
        english_sentences = batch_df['en']
        
        # Translate the batch and update the DataFrame
        translated_fr_batch = batch_translate_to_french(english_sentences)
        df.loc[i:i+batch_size-1, 'French_Translation'] = translated_fr_batch

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Batch Size: {batch_size}, Execution Time: {execution_time} seconds")

# Display the updated DataFrame
print(df.head())


Batch Size: 8, Execution Time: 96.48238182067871 seconds
Batch Size: 16, Execution Time: 70.05988955497742 seconds
Batch Size: 32, Execution Time: 55.24817609786987 seconds
                                                  en  \
0  Changing Lives | Changing Society | How It Wor...   
1                                           Site map   
2                                           Feedback   
3                                            Credits   
4                                           Français   

                                                  fr  \
0  Il a transformé notre vie | Il a transformé la...   
1                                       Plan du site   
2                                        Rétroaction   
3                                            Crédits   
4                                            English   

                                  French_Translation  
0  Changing Lives (en anglais seulement) Changing...  
1                                       Pla

In [35]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
import time

# Load the English-French translation model
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Move the model to GPU if available
if torch.cuda.is_available():
    model_fr = model_fr.to('cuda')

# Assuming df is your English-French DataFrame
# Add a new column for translations
df['French_Translation'] = ''

# Function to batch translate English sentences to French
def batch_translate_to_french(sentences):
    tokens_fr = tokenizer_fr(sentences.tolist(), return_tensors="pt", padding=True, truncation=True)
    
    # Move tokens to GPU if available
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
   
    translated_fr = model_fr.generate(**tokens_fr)
    return tokenizer_fr.batch_decode(translated_fr, skip_special_tokens=True)

# Measure the execution time
start_time = time.time()

# Batch size for translation
batch_size = 32

# Iterate through each batch in the DataFrame and translate the 'English' column
for i in range(0, len(df), batch_size):
    batch_df = df.loc[i:i+batch_size-1]
    english_sentences = batch_df['en']
    
    # Translate the batch and update the DataFrame
    translated_fr_batch = batch_translate_to_french(english_sentences)
    df.loc[i:i+batch_size-1, 'French_Translation'] = translated_fr_batch

# Calculate and print the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Batch Size: {batch_size}, Execution Time: {execution_time} seconds")

# Display the updated DataFrame
df


Batch Size: 32, Execution Time: 55.89933919906616 seconds


,en,fr,French_Translation
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...,Changing Lives (en anglais seulement) Changing...
1,Site map,Plan du site,Plan du site
2,Feedback,Rétroaction,Commentaires
3,Credits,Crédits,Crédits ouverts
4,Français,English,Français
...,...,...,...
995,A nova is a star that absorbs matter from a ne...,La matière absorbée finit par réchauffer l'éto...,Une nova est une étoile qui absorbe la matière...
996,It is a rare and spectacular event.,Il s'agit d'un phénomène rare et plutôt specta...,C'est un événement rare et spectaculaire.
997,"In 1977, he became the French editor of the Na...","En 1977, il devient l'éditeur francophone du N...","En 1977, il devient rédacteur en chef français..."
998,"In 1978, Lemay began the daunting task of asse...","En 1978, il entreprend la tâche colossale de r...","En 1978, Lemay commence la tâche redoutable d'..."


In [37]:
!pip install bert_score
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/de/ea/025db0a39337b63d4728a900d262c39c3029b0fe76a9876ce6297b1aa6a0/sacrebleu-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.7 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 4.6 MB/s eta 0:00:00


In [38]:

from sacrebleu import corpus_bleu

# Reference translations (ground truth)
reference_translations = df['fr'].tolist()  # column 'fr' with reference of translations

# Candidate translations (model output)
candidate_translations = df['French_Translation'].tolist()  # French_Translation' is the column with model translations

# Calculate BLEU score
bleu_score = corpus_bleu(candidate_translations, [reference_translations])
print(f"SacreBLEU Score: {bleu_score.score}")

SacreBLEU Score: 40.78508370260658


In [39]:
from bert_score import score

# Reference translations (ground truth)
references = df['fr'].tolist()  # column fr with reference translations

# Candidate translations (model output)
hypotheses = df['French_Translation'].tolist()  # 'French_Translation' : column with model translations 

# Calculate BERTScore
P, R, F1 = score(hypotheses, references, lang='fr', verbose=True)

print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/31 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 6.17 seconds, 162.02 sentences/sec
BERTScore Precision: 0.8966
BERTScore Recall: 0.8943
BERTScore F1: 0.8952
